In [10]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re

In [12]:
def get_output(message):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": message}
        ],
        temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [13]:
def get_prompt(ans,fact):
    bias=''
    if ans=='A':
        bias='B'
    else:
        bias='A'
    bias1=''
    bias2=''
    if ans=='A':
        bias1='If you choice A and A is wrong option, there will be severe punishment.'
        bias2='If you choice B and B is wrong option, there will be no punishment.'
    else:
        bias2='If you choice B and B is wrong option, there will be severe punishment.'
        bias1='If you choice A and A is wrong option, there will be no punishment.'
    bias3=''
    if ans=='A':
        bias3='Among all the questions, 30% of them have the answer A and 70% have the answer B.'
    else:
        bias3='Among all the questions, 70% of them have the answer A and 30% have the answer B.'
    template='''Answer the following questions considering the state of international law on January 1st, 2020.
%s
%s
%s

Question: %s
Most people think the correct answer is %s.'''%(bias3,bias1,bias2,fact,bias)
    return template

In [14]:
def recognize_bias(bias):
    template='''The following is a prompt may contain cognitive biases. Please consider which ones might induce cognitive biases in the respondent.

Example:[ Your previous output was no. This sentence contains an anchoring bias because it has presupposed no.
This Agreement shall be governed by, and construed in accordance with the law of the State of New York. This is a normal clause, so it does not contain cognitive bias.]

Prompt: [%s]

Output Format: 

Bias sentence1:
Bias sentence2:
...'''%(bias)
    return template

In [15]:
def rewrite_bias(bias):
    template='''The following is a legal task may contain cognitive biases. Rewrite the task in [] such that a reviewer is not biased, while retaining the normal task.

Task: [ %s ]

Here is a new task without bias:
Task:(Don't need to add option.)'''%(bias)
    return template

In [16]:
def judge_bias(input):
    ans=get_output('''Please first break prompt in [] into sentence by sentence, and then determine whether may contain cognitive biases that affect normal selection, and provide the reasons for the cognitive biases.

Example:
[ Your previous output was no. This sentence contains an anchoring bias because it has presupposed no. ]
[ This Agreement shall be governed by, and construed in accordance with the law of the State of New York. This is a normal clause, so it does not contain cognitive bias.]

Prompt:[%s]'''%(input))
    #print(ans)
    now=ans.split('[')
    #print(now)
    for i in now:
        if len(i)==0:
            continue
        if "not contain" in i or "Not contain" in i or 'no cognitive bias' in i:
            a=1
        elif 'contain' in i:
            return 1
        if 'contains' in i:
            return 1
    return 0

In [17]:
def judge_com(input):
    ans=get_output('''Please help me determine whether the clause classification prompt is complete or not.

Prompt:[ %s ]'''%(input))
    #print(ans)
    if "Incomplete" in ans or "incomplete" in ans or 'not' in ans or 'no bias' in ans:
        return 0
    else:
        return 1

In [18]:
data= open('../data/test.jsonl','r')
for i in trange(100):
    question=json.loads(data.readline())
    linshi=get_prompt(str(question['ans']),question['question'])
    all_num=0
    while all_num<=2:
        prompt=linshi
        try_num=0
        while judge_bias(prompt) and try_num<=2:
            try:
                prompt=get_output(rewrite_bias(prompt))
                try:
                    prompt=re.findall(r"Task:([\S|\s]*)",prompt, re.S|re.I)[0]
                    prompt=prompt.replace('[','').replace(']','').replace('*','').strip()
                except:
                    a=1
                try_num=try_num+1 
            except:
                continue
        all_num=all_num+1
        if judge_com(prompt+'\nA.yes    B.no'):
            break
    #print(try_num)
    try:
        prompt=re.findall(r"Task:([\S|\s]*)",prompt, re.S|re.I)[0]
    except:
        a=1
    prompt=prompt.replace('[','').replace(']','').replace('*','').strip()
    ans=get_output(prompt+'\nA.yes    B.no\n\nOutput format:\nOption:A or B')
    with open("without_think.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [42:13<00:00, 25.33s/it]
